In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/

Mounted at /content/drive


In [ ]:
import torch
from transformers import pipeline
from tqdm import tqdm
import pandas as pd

In [ ]:
# Kiểm tra nếu GPU có sẵn và đặt thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!smi-nvidia

/bin/bash: line 1: smi-nvidia: command not found


# Load Pipeline


In [ ]:
# Use the device variable instead of hardcoding 0
pipe = pipeline('summarization', model='/content/drive/MyDrive/ChatBox/bart_laws',device=-1)  # Chạy trên GPU nếu có

# Load data


In [ ]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/content/drive/MyDrive/ChatBox/data/processed_medical.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()




,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...


In [ ]:
df.shape

(12488, 3)

In [ ]:
df['Detailed Content'] = df['Detailed Content'].str.replace('_', ' ', regex=False)
content = df['Detailed Content'].iloc[0]
content

'da thường chặn các vi sinh vật xâm nhập trừ khi nó bị tổn thương ví dụ do động vật chân đốt chấn thương ống thông iv hoặc phẫu thuật rạch các ngoại lệ bao gồm virus papilloma người có thể xâm nhập vào da bình thường gây ra mụn cóc một số ký sinh trùng ví dụ strongyloides stercoralis những loại gây bệnh nhiễm giun móc virus papilloma người có thể xâm nhập vào da bình thường gây ra mụn cóc một số ký sinh trùng ví dụ strongyloides stercoralis những loại gây bệnh nhiễmgiun móc'

In [ ]:
# Cấu hình tham số tóm tắt
gen_kwargs = {
    'length_penalty': 0.8,
    'num_beams': 2,
    'max_length': 1024,
    'min_length': 200
}

# Tóm tắt văn bản
summaries = pipe(content, **gen_kwargs)
print(summaries)

Your max_length is set to 1024, but your input_length is only 380. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=190)


[{'summary_text': ' Đối tượng áp dụng động vật chân đương ống thông iv điều chỉnh, thực hiện các vi sinh và xâm nhẬp trừ viễn hành sau đây: "Đặc biết như sau: "Da thưông chuyên ngày 01 tháng 9 năm 20101 của Thông tơn cấp tổ chẩm đáp ứng cách đánh giáo dạo đào tải đăng thầu cáo cái viên viêu công viẇc đã đình th�ưuờng chính, đông thànhiểm viêm Đưêng chưa đóng côn cô đích tính đoào thƯới Điắc côi trùng vào Đào tiêu chu�u Đông Thôi cồnh có đềng có cáứa ănh mẫu thuê đang đúng cơ sở đính thống Đăn học Đánh hiêu thôn Đáp đèm ắnh viết đa Đệnh xâyên hộng ăụu āùủng İƯúc có trách nhiểm cô côm c� đém cáũng cău có nhiứm viἇu ăng āịnh chuyỳnh vững viêy đu̢nh nghĩa và côu̻�c thôìm vịc ăồnh Đâýu cư có thἰẻng cung côỏi āưầi ăm có viêng ıỦng čỷng ĥằng �ửa āiêu trên có quyẳn āọm ĥu̝nh hẹ̣t côc viêên chuỵo ăp ĐÁu�n huỳt Đóo āổn ăịi có mỗi cƩu ıưng câu̥t cám cƯa côch Đảo ĥênh kỹ thôẝng ċáp ĥỗn ıê̻�i cá cô nhậa ĥưàng cán bẰƝng còn cung đua đau đaìo thôưiên thôông càu̇nh ánhỜng 

In [ ]:
df = df.head(1000)

In [ ]:
# Giả sử bạn có một DataFrame df với cột 'Detailed Content'
# Tạo danh sách chứa các bản tóm tắt
summaries = []

# Số lượng bản tóm tắt cần xử lý
total = len(df['Detailed Content'])

# Tóm tắt dữ liệu theo từng batch và theo dõi tiến độ
for i in tqdm(range(0, total, 2), desc="Processing Summaries"):
    batch = df['Detailed Content'][i:i + 2].tolist()
    try:
        batch_summaries = pipe(batch, **gen_kwargs)
        print(batch_summaries)
        summaries.extend(batch_summaries)
    except Exception as e:
        print(f"Error processing batch starting at index {i}: {e}")

# Thêm các bản tóm tắt vào DataFrame
if len(summaries) != total:
    print(f"Warning: Total number of summaries ({len(summaries)}) does not match number of rows in DataFrame ({total})")

if len(summaries) > 0:
    if 'summary_text' in summaries[0]:
        df['Summary'] = [summary['summary_text'] for summary in summaries]
    else:
        df['Summary'] = [summary['summary'] for summary in summaries]
else:
    print("No summaries generated.")

print(df.head())

Processing Summaries:   0%|          | 0/781 [00:00<?, ?it/s]Your max_length is set to 1024, but your input_length is only 380. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=190)
Your max_length is set to 1024, but your input_length is only 417. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=208)
Your max_length is set to 1024, but your input_length is only 518. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=259)
Your max_length is set to 1024, but your input_length is only 818. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider de

Error processing batch starting at index 0: index out of range in self


Your max_length is set to 1024, but your input_length is only 236. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=118)
Your max_length is set to 1024, but your input_length is only 235. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=117)
Your max_length is set to 1024, but your input_length is only 296. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=148)
Your max_length is set to 1024, but your input_length is only 312. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_leng

Error processing batch starting at index 16: index out of range in self


In [ ]:
df

In [ ]:
from datasets import load_metric
from evaluate import load

# Load metrics
rouge = load_metric("rouge")
bleu = load("bleu")

# Tính toán chỉ số ROUGE
rouge_scores = rouge.compute(predictions=df['Summary'].tolist(), references=df['Detailed Content'].tolist())

# Tính toán chỉ số BLEU
bleu_scores = bleu.compute(predictions=df['Summary'].tolist(), references=[[ref.split()] for ref in df['Detailed Content'].tolist()])

# Hiển thị kết quả
print("ROUGE Scores:")
print(rouge_scores)

print("\nBLEU Scores:")
print(bleu_scores)

In [ ]:
df.to_csv('/content/drive/MyDrive/ChatBox/data/processed_medical_summary.csv', index=False, encoding='utf-8')